# Escuelas de Cienias Informaticas #32

## Arquitecturas Avanzadas de Cómputo

## Profesor Invitado, Javier Navaridas Palma

### Trabajo final: Simulador de Cache L1

- Mascitti, Julio Augusto - 954/11 - mascittija@gmail.com


El siguiente trabajo esta enfocado en simular y experimentar los accesos a memoria que ocurren en una computadora al realizar lecturas de memoria, proceso fundamental para el funcionamiento de cualquier arquitectura que sigue el modelo Von-Neuman.
Para lo mismo, se implementaran una memoria principal, la cula será accedida por distintas rutinas para probar la performance de dichos accesos y luego se introducirá una memoria cache L1 para intentar mejorar dicha performance.



https://campus.exactas.uba.ar/pluginfile.php/93653/mod_resource/content/1/uArch-mem.pdf

In [15]:
import random as rn
import time as tm

LINESIZE = 8

class Address:
    def __init__(self, address):
        self.address = address
        self.offset = (address % LINESIZE)
        self.base = address - self.offset
        
class CacheLine:
    def __init__(self, address, value, lineNumber):
        self.dirty = False
        self.addressLine = address 
        self.time = tm.time()
        self.lineNumber = lineNumber
        self.data = value

    def get_value(self, offset):
        return self.data[offset]
    
    def set_value(self, offset, value):
        self.dirty = True
        self.data[offset] = value
        
    def replace_line(self, address, value):
        self.dirty = False
        self.addressLine = address 
        self.time = tm.time()
        self.data = value
        return self.time
    
    def get_number(self):
        return self.lineNumber
    
    def get_time(self):
        return self.time
    
    def tic(self):
        self.time = tm.time()
    
    def is_dirty(self):
        return self.dirty
    
    def get_data(self):
        return sel.data
        
class Memory:
    
    def __init__(self, memorySize, memorySpeed, timeTest):
        self.memorySize = 2**memorySize
        self.speed = memorySpeed
        self.timeTest = timeTest
        self.data = [rn.randint(0,2**7) for i in range(self.memorySize)]
#        if lineSize > memorySize:
#            raise NameError('Size of line is greater than memory available')
#        self.lineSize = 2**lineSize
    
    def read(self, memAddress):
        if memAddress > self.memorySize:
            raise IndexError
        self.wait()
        return self.data[memAddress]
    
    def write(self, memAddress, value):
        if address > self.memorySize:
            raise IndexError
        self.wait()
        self.data[memAddress] = value
        
    def read_line(self, address):
        self.wait()
        return self.data[address.base : address.base + LINESIZE]
    
    def write_line(self, address, value):
        tm.sleep(self.speed)
        self.data[address.base : address.base + LINESIZE] = value
        
    def wait(self):
        if self.timeTest:
            tm.sleep(self.speed)
            print('MEM')

class CacheMemory:
    
    FIFO = 'FIFO'
    RAND = 'RAND'
    LRU = 'LRU'
    LFU = 'LFU'
    
    COPY_BACK = 'CB'
    WRITE_THROUGH = 'WT'
    
    def __init__(self, mainMemorySize, cacheSize, replacementPol, writePol, cacheSpeed, penaltySpeed, timeTest):
        self.mainMemory = Memory(mainMemorySize, cacheSpeed * penaltySpeed, timeTest)
        self.cacheSize = 2**cacheSize
        self.replPol = replacementPol
        self.writePol = writePol
        self.timeTest = timeTest
        
        self.freeLines = self.cacheSize
        
        self.cacheSpeed = cacheSpeed
        
        self.cachedLines = dict()
        
        self.addressByLine = [None] * self.cacheSize
        self.timeByLine = [None] * self.cacheSize
        self.usesByLine = [0] * self.cacheSize
        
        self.HIT = 0
        self.MISS = 0
        
        self.nextLineRemove = 0
        
    def write(self, address, value):
        if self.cacheSize == 1:
            self.mainMemory.write(address, value)
            return True

        a = Address(address)
        
        if a.base in self.cachedLines:
            self.wait()
            line = self.cachedLines[a.base]
            line.tic()
            self.timeByLine[line.get_number()] = line.get_time()
            self.usesByLine[line.get_number()] = self.usesByLine[line.get_number()] + 1
            line.set_value(a.offset, value)

            if self.writePol == self.COPY_BACK:
                self.mainMemory.write(address, value)
            return True
        else:
            print('DESALOJAR')
            line = self.replace_data(a.base, memoryData)
            
        return True
        
    def wait(self):
        if self.timeTest:
            tm.sleep(self.cacheSpeed)
            #print('CACH')
        
    def read(self, address):
        if self.cacheSize == 1:
            return self.mainMemory.read(address)
        
        a = Address(address)
        
        if a.base in self.cachedLines:
            self.wait()
            print('HIT')
            self.HIT = self.HIT + 1
            line = self.cachedLines[a.base]
            line.tic()
            self.timeByLine[line.get_number()] = line.get_time()
            self.usesByLine[line.get_number()] = self.usesByLine[line.get_number()] + 1
            return line.get_value(a.offset)
        
        self.MISS = self.MISS + 1
        print('MISS')
        
        memoryData = self.mainMemory.read_line(a)
        
        if self.freeLines > 0:
            lineNumber = self.cacheSize - self.freeLines
            #print('cachesize: %, FreeLines: %, lineNumber: %', self.cacheSize, self.freeLines, lineNumber)
            self.freeLines = self.freeLines - 1
            self.addressByLine[lineNumber] = a.base
            line = CacheLine(a.base, memoryData, lineNumber)
            self.timeByLine[line.get_number()] = line.get_time()
            self.usesByLine[line.get_number()] = self.usesByLine[line.get_number()] + 1
            self.cachedLines[a.base] = line
        else:
            print('DESALOJAR')
            line = self.replace_data(a.base, memoryData)
        
        return line.get_value(a.offset)
            
    def replace_data(self, address, data):
        if self.replPol == self.FIFO:
            oldBase = self.addressByLine[self.nextLineRemove]
            self.addressByLine[self.nextLineRemove] = address
            newLine = CacheLine(address, data, self.cachedLines[oldBase].get_number())
            self.timeByLine[newLine.get_number()] = newLine.get_time()
            self.cachedLines[address] = newLine
            
            if self.writePol == self.WRITE_THROUGH and self.cachedLines[oldBase].is_dirty():
                self.mainMemory.write_line(address, self.cachedLines[oldBase].get_data())

            del self.cachedLines[oldBase]
            self.nextLineRemove = (self.nextLineRemove + 1) % self.cacheSize
            return newLine
        
        if self.replPol == self.RAND:
            self.nextLineRemove = rn.randint(0, self.cacheSize - 1)
            oldBase = self.addressByLine[self.nextLineRemove]
            self.addressByLine[self.nextLineRemove] = address
            newLine = CacheLine(address, data, self.cachedLines[oldBase].get_number())
            self.timeByLine[newLine.get_number()] = newLine.get_time()
            self.cachedLines[address] = newLine

            if self.writePol == self.WRITE_THROUGH and self.cachedLines[oldBase].is_dirty():
                self.mainMemory.write_line(address, self.cachedLines[oldBase].get_data())
            
            del self.cachedLines[oldBase]
            return newLine
        
        if self.replPol == self.LRU:
            self.nextLineRemove = self.next_LRU()
            oldBase = self.addressByLine[self.nextLineRemove]
            self.addressByLine[self.nextLineRemove] = address
            newLine = CacheLine(address, data, self.cachedLines[oldBase].get_number())
            self.timeByLine[newLine.get_number()] = newLine.get_time()
            self.cachedLines[address] = newLine
            
            if self.writePol == self.WRITE_THROUGH and self.cachedLines[oldBase].is_dirty():
                self.mainMemory.write_line(address, self.cachedLines[oldBase].get_data())
            
            del self.cachedLines[oldBase]
            return newLine
        
        if self.replPol == self.LFU:
            self.nextLineRemove = self.next_LFU()
            oldBase = self.addressByLine[self.nextLineRemove]
            self.addressByLine[self.nextLineRemove] = address
            newLine = CacheLine(address, data, self.cachedLines[oldBase].get_number())
            self.timeByLine[newLine.get_number()] = newLine.get_time()
            self.cachedLines[address] = newLine

            if self.writePol == self.WRITE_THROUGH and self.cachedLines[oldBase].is_dirty():
                self.mainMemory.write_line(address, self.cachedLines[oldBase].get_data())
            
            del self.cachedLines[oldBase]
            return newLine

    def next_LRU(self):
        lineNumber = 0
        minTime = self.timeByLine[lineNumber]
        for i in range(1, len(self.timeByLine)):
            if self.timeByLine[i] < minTime:
                minTime = self.timeByLine[i]
                lineNumber = i
        print('Linea: %', lineNumber)
        return lineNumber

    def next_LFU(self):
        lineNumber = 0
        minUsed = self.usesByLine[lineNumber]
        for i in range(1, len(self.usesByLine)):
            if self.usesByLine[i] < minUsed:
                minUsed = self.usesByLine[i]
                lineNumber = i
            self.usesByLine[i] = self.usesByLine[i] / 2
        print('Linea: %', lineNumber)
        return lineNumber
    

In [13]:
m = CacheMemory(20, 3, 'LFU', 'CB', 0.1, 10, False)


for i in range(0,100,3):
    m.read(i)


MISS
HIT
HIT
MISS
HIT
HIT
MISS
HIT
MISS
HIT
HIT
MISS
HIT
HIT
MISS
HIT
MISS
HIT
HIT
MISS
HIT
HIT
MISS
DESALOJAR
('Linea: %', 2)
HIT
MISS
DESALOJAR
('Linea: %', 1)
HIT
HIT
MISS
DESALOJAR
('Linea: %', 3)
HIT
HIT
MISS
DESALOJAR
('Linea: %', 2)
HIT
MISS
DESALOJAR
('Linea: %', 1)
HIT
